In [2]:
# 🎶 Importar Magenta e utilitários
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from magenta.music import sequence_proto_to_midi_file, NoteSequence, note_seq
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Carregar o modelo Magenta pré-treinado
BUNDLE_PATH = 'basic_rnn.mag'  # Certifique-se de que esse arquivo está no seu diretório
bundle = sequence_generator_bundle.read_bundle_file(BUNDLE_PATH)
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['basic_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

# Criar uma melodia inicial (primer melody)
primer_sequence = NoteSequence()
primer_sequence.tempos.add(qpm=120)

# Notas: C4, E4, G4
notas = [60, 64, 68]
tempo_por_nota = 0.5
for i, nota in enumerate(notas):
    note = primer_sequence.notes.add()
    note.pitch = nota
    note.start_time = i * tempo_por_nota
    note.end_time = (i + 1) * tempo_por_nota
    note.velocity = 40
primer_sequence.total_time = len(notas) * tempo_por_nota

# Configurar parâmetros de geração
from note_seq.protobuf import generator_pb2


generator_options = generator_pb2.GeneratorOptions()
generate_section = generator_options.generate_sections.add()
generate_section.start_time = primer_sequence.total_time
generate_section.end_time = primer_sequence.total_time + 20  # 20 segundos de música

# Gerar a melodia
print("Gerando melodia...")
sequence = melody_rnn.generate(primer_sequence, generator_options)

# Salvar como arquivo MIDI
output_path = '../midis/melodia_magenta.mid'
sequence_proto_to_midi_file(sequence, output_path)
print(f"Melodia salva como {output_path}")

ModuleNotFoundError: No module named 'magenta'

In [ ]:
import tkinter as tk
from tkinter import messagebox
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from magenta.music import sequence_proto_to_midi_file, NoteSequence, note_seq
from note_seq.protobuf import generator_pb2
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Carregar modelo
BUNDLE_PATH = 'basic_rnn.mag'
bundle = sequence_generator_bundle.read_bundle_file(BUNDLE_PATH)
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['basic_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

# Função de geração
def gerar_melodia():
    try:
        notas_str = entry_notas.get()
        notas = [int(n) for n in notas_str.split(',')]
        tempo_por_nota = float(entry_tempo.get())
        duracao = float(entry_duracao.get())
        velocidade = int(entry_velocidade.get())
        saida = entry_saida.get()

        # Criar sequência inicial
        primer_sequence = NoteSequence()
        primer_sequence.tempos.add(qpm=120)

        for i, nota in enumerate(notas):
            note = primer_sequence.notes.add()
            note.pitch = nota
            note.start_time = i * tempo_por_nota
            note.end_time = (i + 1) * tempo_por_nota
            note.velocity = velocidade
        primer_sequence.total_time = len(notas) * tempo_por_nota

        # Parâmetros de geração
        generator_options = generator_pb2.GeneratorOptions()
        generate_section = generator_options.generate_sections.add()
        generate_section.start_time = primer_sequence.total_time
        generate_section.end_time = primer_sequence.total_time + duracao

        # Gerar e salvar
        sequence = melody_rnn.generate(primer_sequence, generator_options)
        sequence_proto_to_midi_file(sequence, saida)
        messagebox.showinfo("Sucesso", f"Melodia salva como {saida}")
    except Exception as e:
        messagebox.showerror("Erro", str(e))

# GUI
root = tk.Tk()
root.title("Gerador de Melodias")

tk.Label(root, text="Notas iniciais (ex: 60,64,67):").grid(row=0, column=0, sticky="e")
entry_notas = tk.Entry(root, width=30)
entry_notas.insert(0, "60,64,67")
entry_notas.grid(row=0, column=1)

tk.Label(root, text="Tempo por nota (s):").grid(row=1, column=0, sticky="e")
entry_tempo = tk.Entry(root)
entry_tempo.insert(0, "0.5")
entry_tempo.grid(row=1, column=1)

tk.Label(root, text="Duração da geração (s):").grid(row=2, column=0, sticky="e")
entry_duracao = tk.Entry(root)
entry_duracao.insert(0, "20")
entry_duracao.grid(row=2, column=1)

tk.Label(root, text="Velocidade da nota (0-127):").grid(row=3, column=0, sticky="e")
entry_velocidade = tk.Entry(root)
entry_velocidade.insert(0, "40")
entry_velocidade.grid(row=3, column=1)

tk.Label(root, text="Caminho do arquivo MIDI:").grid(row=4, column=0, sticky="e")
entry_saida = tk.Entry(root, width=30)
entry_saida.insert(0, "../midis/melodia_magenta.mid")
entry_saida.grid(row=4, column=1)

tk.Button(root, text="Gerar Melodia 🎼", command=gerar_melodia, bg="lightgreen").grid(row=5, column=0, columnspan=2, pady=10)

root.mainloop()


'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from C:\Users\dpereira\AppData\Local\Temp\tmpfd_qgobx\model.ckpt
